#### Training a neural network to estimate selection coefficients
#### Seth Temple, sdtemple.github.io
#### 11/11/2024

A decent amount of code is Copy + Paste and modify from: 

https://github.com/benmoseley/harmonic-oscillator-pinn/blob/main/Harmonic%20oscillator%20PINN.ipynb

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from simple_selcoef import * # must be in same folder

In [ ]:
### population genetics

# simulation data
num_rep = 10
s_step = 0.002 # step size for selection coefficient
num_s = 50 # numbers of steps for selection coefficient
p_start = 0.1 # starting frequency in loop
p_step = 0.02 # step size for frequency
num_p = 40 # number of steps for frequency
first = 50 # use this many generations as input data

# make an Ne file
make_constant_Ne('c10.ne',1e4, 500)
Ne = read_Ne('c10.ne')

### neural networks

num_epochs = 20
num_batches = 10
num_hidden = 32
num_layers = 3
act = nn.Tanh


In [ ]:
class FCN(nn.Module):
    "Defines a connected network"
    
    def __init__(self, N_INPUT, N_OUTPUT, N_HIDDEN, N_LAYERS, activation):
        super().__init__()
        self.fcs = nn.Sequential(*[
                        nn.Linear(N_INPUT, N_HIDDEN),
                        activation()])
        self.fch = nn.Sequential(*[
                        nn.Sequential(*[
                            nn.Linear(N_HIDDEN, N_HIDDEN),
                            activation()]) for _ in range(N_LAYERS-1)])
        self.fce = nn.Linear(N_HIDDEN, N_OUTPUT)
        
    def forward(self, x):
        x = self.fcs(x)
        x = self.fch(x)
        x = self.fce(x)
        return x

In [ ]:
# uncomment to create data
ss = [s_step * i for i in range(num_s)]
N = num_rep
ps = [p_start + i * p_step for i in range(num_p)]
M = N*len(ss)*len(ps)
y_array = np.zeros((M,1),dtype=np.float32)
x_array = np.zeros((M,first),dtype=np.float32)
# j = 0
# for p in ps:
#     for s in ss:
#         for n in range(N):
#             y = s
#             x0, _, _ = walk_variant_backward(y,p,Ne,random_walk=True)
#             x = x0[:first]
#             x_array[j,:] = x
#             y_array[j] = y
#             j += 1
# x_data = torch.from_numpy(x_array)
# y_data = torch.from_numpy(y_array)
# torch.save(x_data,'x_tensor.pth')
# torch.save(y_data,'y_tensor.pth')


In [112]:
# train standard neural network to fit training data

x_data = torch.load('x_tensor.pth')
y_data = torch.load('y_tensor.pth')

dataset = TensorDataset(x_data,y_data)
dataloader = DataLoader(dataset, batch_size=num_batches, shuffle=True)
torch.manual_seed(123)
model = FCN(first,1,num_hidden,num_layers,act)
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)
files = []
for i in range(num_epochs):
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        yh = model(inputs)
        loss = torch.mean((yh-targets)**2)# use mean squared error
        loss.backward()
        optimizer.step()
            

/tmp/ipykernel_36315/3736850351.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  x_data = torch.load('x_tensor.pth')
/tmp/ipykernel_36315/3736850351.py:4: FutureWarning: 

In [ ]:
# make a prediction
p = 0.5
s = 0.05
x0, _, _ = walk_variant_backward(s,p,Ne,random_walk=True)
x = x0[:first]
x_t = torch.from_numpy(x).to(torch.float32)
model(x_t)

tensor([0.0455], grad_fn=<ViewBackward0>)